This 3A notebook will be perform the second half of NLP preprocessing using spaCy, mostly.  Also, this will produce a large dataframe two lyric sets (with/with out stopwords), each with word counts and spaCy .doc (vectorization, POS tags, Named Entity recognition, etc.).

[NEED TO RUN AFFINITY (AFINN) TO MARRY UP WITH NB 3.]

Pull down the dataset which completed some, but not all of the NLP preprocessing in NB 3 (Using the Fake News group's pre-processing as a guide (https://github.com/georgetown-analytics/From-Russia-With-Love-fake-news-/blob/master/Notebooks/Step_1_Data_Cleaning.ipynb)). 

This particular set has been through: lowercase, remove URLs, ww, special characters, extra whitespace, accented characters and expanded contractions.

Will run spaCy and word counts as is.  Then will remove stopwords. Then rerun spaCy pipeline and count the words.

In [1]:
import s3fs
import pandas as pd
pd.set_option('display.max_columns', 100000)
pd.set_option('display.max_row', 1000000)
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
# import tldextract """Accurately separate the TLD from the registered domain and subdomains of a URL"""
from tqdm.autonotebook import tqdm
tqdm.pandas(desc="progress-bar", leave=False)
import string

import spacy
from spacy.lang import punctuation
from spacy.lang.en import English
from spacy import displacy
nlp = spacy.load("en_core_web_lg")

import unicodedata  # might need to pip install unicodedate2 on aws sagemaker
import contractions
from contractions import contractions_dict ## pip installed this
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import STOPWORDS
import warnings
from afinn import Afinn
warnings.filterwarnings('ignore')

%matplotlib inline
sns.set(style='darkgrid',palette='Dark2',rc={'figure.figsize':(9,6),'figure.dpi':90})

punctuation = string.punctuation + '”' + '“' + '–' + '““' + "’’" + '”'
stopword = stopwords.words('english')
stopwords = set(STOPWORDS)
wordnet_lemmatizer = WordNetLemmatizer()

<ipython-input-1-4fce9e1e4886>:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
fs = s3fs.S3FileSystem(anon=False,key='###',secret='###')

g_df = pd.read_csv('s3://music-lyrics-chain/genres_midcln_df.csv')

In [3]:
g_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86391 entries, 0 to 86390
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   original_csv  86391 non-null  object 
 1   artist_name   86391 non-null  object 
 2   song_name     86391 non-null  object 
 3   link          86391 non-null  object 
 4   lyrics        86391 non-null  object 
 5   language      86391 non-null  object 
 6   genre         86391 non-null  object 
 7   date          0 non-null      float64
dtypes: float64(1), object(7)
memory usage: 5.3+ MB


In [8]:
g_df.shape

(86391, 8)

In [5]:
g_df.iloc[84304]['lyrics']#genres Taylor Swift The Story of Us

'i used to think one day we would tell the story of us. how we met and sparks flew instantly. people would say they are the lucky ones. i used to know my place was a spot next to you. now I am searching the room for an empty seat. because lately i do not even know what page you are on. oh. a simple complication, miscommunication. has lead to fallout. too many things that i wish you knew. so many walls up i can not break through. now I am standing alone in a crowded room. and we are not speaking. and I am dying to know. is it killing you like it is killing me?. and i do not know what to say. since the twist of fate. when it all broke down. and the story of us looks a lot like a tragedy now. next chapter. how did we end up this way?. se me nervously pulling at my clothes. and trying to look busy. and you are doing your best to avoid me. I am starting to think one day I will tell the story of us. how i was losing my mind when i saw you here. but you held your pride like you should have he

In [14]:
lyrics = [_ for _ in g_df['lyrics']]

#def set_doc(lyrics):
    #scores = []
    #for t in lyrics:
        #doc = nlp(t)
        #scores.append(doc)
    #return scores

new_doc = list(nlp.pipe(lyrics))[0]

g_df['spaCy_Doc'] = new_doc

%time

ValueError: Length of values (188) does not match length of index (86391)

nlp.pipe(text) didn't work - didn't match the amount of inputs?  Hmmm.  Sure this is more a python problem on my end vice a problem in spaCy.

In [19]:
lyrics = [_ for _ in g_df['lyrics']]

def set_doc(lyrics):
    scores = []
    for t in lyrics:
        doc = nlp(t)
        scores.append(doc)
    return scores

new_doc = set_doc(g_df['lyrics'].tolist())

g_df['spaCy_Doc'] = new_doc

%time

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.29 µs


In [20]:
g_df.shape

(86391, 9)

In [21]:
g_df.head()

,original_csv,artist_name,song_name,link,lyrics,language,genre,date,spaCy_Doc
0,genres_csv,10000-maniacs,More Than This,/10000-maniacs/more-than-this.html,i could feel at the time. there was no way of ...,ENGLISH,Rock,NaN,"(i, could, feel, at, the, time, ., there, was,..."
1,genres_csv,10000-maniacs,Because The Night,/10000-maniacs/because-the-night.html,"take me now, baby, here as i am. hold me close...",ENGLISH,Rock,NaN,"(take, me, now, ,, baby, ,, here, as, i, am, ...."
2,genres_csv,10000-maniacs,These Are Days,/10000-maniacs/these-are-days.html,these are. these are days you will remember. n...,ENGLISH,Rock,NaN,"(these, are, ., these, are, days, you, will, r..."
3,genres_csv,10000-maniacs,A Campfire Song,/10000-maniacs/a-campfire-song.html,"a lie to say, ""of my mountain has coal veins a...",ENGLISH,Rock,NaN,"(a, lie, to, say, ,, "", of, my, mountain, has,..."
4,genres_csv,10000-maniacs,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,trudging slowly over wet sand. back to the ben...,ENGLISH,Rock,NaN,"(trudging, slowly, over, wet, sand, ., back, t..."


In [22]:
g_df.iloc[84304]['spaCy_Doc']#genres Taylor Swift The Story of Us

i used to think one day we would tell the story of us. how we met and sparks flew instantly. people would say they are the lucky ones. i used to know my place was a spot next to you. now I am searching the room for an empty seat. because lately i do not even know what page you are on. oh. a simple complication, miscommunication. has lead to fallout. too many things that i wish you knew. so many walls up i can not break through. now I am standing alone in a crowded room. and we are not speaking. and I am dying to know. is it killing you like it is killing me?. and i do not know what to say. since the twist of fate. when it all broke down. and the story of us looks a lot like a tragedy now. next chapter. how did we end up this way?. se me nervously pulling at my clothes. and trying to look busy. and you are doing your best to avoid me. I am starting to think one day I will tell the story of us. how i was losing my mind when i saw you here. but you held your pride like you should have hel

In [23]:
!touch g_spaCy1_df.csv
g_df.to_csv('g_spaCy1_df.csv', index= False)
!aws s3 cp g_spaCy1_df.csv s3://music-lyrics-chain/g_spaCy1_df.csv
%time

upload: ./g_spaCy1_df.csv to s3://music-lyrics-chain/g_spaCy1_df.csv
CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 2.86 µs


In [24]:
%%time
# word counts
g_df['word_count'] = g_df["lyrics"].apply(lambda x: len(str(x).split(" ")))

# Character counts
g_df['character_count'] = g_df["lyrics"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))

#average word length
g_df['avg_word_length'] = g_df['character_count'] / g_df['word_count']

CPU times: user 3.23 s, sys: 80.6 ms, total: 3.31 s
Wall time: 3.4 s


In [25]:
g_df.head()

,original_csv,artist_name,song_name,link,lyrics,language,genre,date,spaCy_Doc,word_count,character_count,avg_word_length
0,genres_csv,10000-maniacs,More Than This,/10000-maniacs/more-than-this.html,i could feel at the time. there was no way of ...,ENGLISH,Rock,NaN,"(i, could, feel, at, the, time, ., there, was,...",154,611,3.967532
1,genres_csv,10000-maniacs,Because The Night,/10000-maniacs/because-the-night.html,"take me now, baby, here as i am. hold me close...",ENGLISH,Rock,NaN,"(take, me, now, ,, baby, ,, here, as, i, am, ....",246,1023,4.158537
2,genres_csv,10000-maniacs,These Are Days,/10000-maniacs/these-are-days.html,these are. these are days you will remember. n...,ENGLISH,Rock,NaN,"(these, are, ., these, are, days, you, will, r...",181,717,3.961326
3,genres_csv,10000-maniacs,A Campfire Song,/10000-maniacs/a-campfire-song.html,"a lie to say, ""of my mountain has coal veins a...",ENGLISH,Rock,NaN,"(a, lie, to, say, ,, "", of, my, mountain, has,...",272,1027,3.775735
4,genres_csv,10000-maniacs,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,trudging slowly over wet sand. back to the ben...,ENGLISH,Rock,NaN,"(trudging, slowly, over, wet, sand, ., back, t...",117,530,4.529915


Creating the _sml version of the lyrics, the spaCy_Doc column and the word counts.

In [26]:
def replace_punctuation(text):
    filters = string.punctuation + '”' + '“' + '–' 
    translate_dict = dict((c, " ") for c in filters)   
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    return text

def stops_letters(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2 and token not in stopword:
            result.append(token)
            
    return " ".join(result)

g_df['lyrics_sml'] =g_df['lyrics'].apply(replace_punctuation)
g_df['lyrics_sml'] =g_df['lyrics'].apply(stops_letters)

%time

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 µs


In [27]:
g_df.head()

,original_csv,artist_name,song_name,link,lyrics,language,genre,date,spaCy_Doc,word_count,character_count,avg_word_length,lyrics_sml
0,genres_csv,10000-maniacs,More Than This,/10000-maniacs/more-than-this.html,i could feel at the time. there was no way of ...,ENGLISH,Rock,NaN,"(i, could, feel, at, the, time, ., there, was,...",154,611,3.967532,feel time way knowing fallen leaves night blow...
1,genres_csv,10000-maniacs,Because The Night,/10000-maniacs/because-the-night.html,"take me now, baby, here as i am. hold me close...",ENGLISH,Rock,NaN,"(take, me, now, ,, baby, ,, here, as, i, am, ....",246,1023,4.158537,baby hold close try understand desire hunger b...
2,genres_csv,10000-maniacs,These Are Days,/10000-maniacs/these-are-days.html,these are. these are days you will remember. n...,ENGLISH,Rock,NaN,"(these, are, ., these, are, days, you, will, r...",181,717,3.961326,days remember promise world warm feel know tru...
3,genres_csv,10000-maniacs,A Campfire Song,/10000-maniacs/a-campfire-song.html,"a lie to say, ""of my mountain has coal veins a...",ENGLISH,Rock,NaN,"(a, lie, to, say, ,, "", of, my, mountain, has,...",272,1027,3.775735,lie mountain coal veins beds dig men axes dig ...
4,genres_csv,10000-maniacs,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,trudging slowly over wet sand. back to the ben...,ENGLISH,Rock,NaN,"(trudging, slowly, over, wet, sand, ., back, t...",117,530,4.529915,trudging slowly wet sand bench clothes stolen ...


In [29]:
g_df.iloc[84304]['lyrics_sml']#genres Taylor Swift The Story of Us

'think day tell story met sparks flew instantly people lucky ones know place spot searching room seat lately know page simple complication lead fallout things wish knew walls break standing crowded room speaking dying know killing like killing know twist fate broke story looks lot like tragedy chapter end way nervously pulling clothes trying look busy best avoid starting think day tell story losing mind saw held pride like held scared ending pretending tell miss know heard silence loud standing crowded room speaking dying know killing like killing know twist fate broke story looks lot like tragedy looking like contest act like care liked better battles hands lay armor love fight things wish knew story ending soon standing crowded room speaking dying know killing like killing know twist fate broke story looks lot like tragedy speaking dying know killing like killing know twist fate going story looks lot like tragedy end'

In [30]:
lyrics = [_ for _ in g_df['lyrics_sml']]

def set_doc(lyrics):
    scores = []
    for t in lyrics:
        doc = nlp(t)
        scores.append(doc)
    return scores

new_doc = set_doc(g_df['lyrics'].tolist())

g_df['spaCy_Doc_sml'] = new_doc

%time

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.1 µs


In [31]:
g_df.head()

,original_csv,artist_name,song_name,link,lyrics,language,genre,date,spaCy_Doc,word_count,character_count,avg_word_length,lyrics_sml,spaCy_Doc_sml
0,genres_csv,10000-maniacs,More Than This,/10000-maniacs/more-than-this.html,i could feel at the time. there was no way of ...,ENGLISH,Rock,NaN,"(i, could, feel, at, the, time, ., there, was,...",154,611,3.967532,feel time way knowing fallen leaves night blow...,"(i, could, feel, at, the, time, ., there, was,..."
1,genres_csv,10000-maniacs,Because The Night,/10000-maniacs/because-the-night.html,"take me now, baby, here as i am. hold me close...",ENGLISH,Rock,NaN,"(take, me, now, ,, baby, ,, here, as, i, am, ....",246,1023,4.158537,baby hold close try understand desire hunger b...,"(take, me, now, ,, baby, ,, here, as, i, am, ...."
2,genres_csv,10000-maniacs,These Are Days,/10000-maniacs/these-are-days.html,these are. these are days you will remember. n...,ENGLISH,Rock,NaN,"(these, are, ., these, are, days, you, will, r...",181,717,3.961326,days remember promise world warm feel know tru...,"(these, are, ., these, are, days, you, will, r..."
3,genres_csv,10000-maniacs,A Campfire Song,/10000-maniacs/a-campfire-song.html,"a lie to say, ""of my mountain has coal veins a...",ENGLISH,Rock,NaN,"(a, lie, to, say, ,, "", of, my, mountain, has,...",272,1027,3.775735,lie mountain coal veins beds dig men axes dig ...,"(a, lie, to, say, ,, "", of, my, mountain, has,..."
4,genres_csv,10000-maniacs,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,trudging slowly over wet sand. back to the ben...,ENGLISH,Rock,NaN,"(trudging, slowly, over, wet, sand, ., back, t...",117,530,4.529915,trudging slowly wet sand bench clothes stolen ...,"(trudging, slowly, over, wet, sand, ., back, t..."


In [32]:
%%time
# word counts
g_df['word_count_sml'] = g_df['lyrics_sml'].apply(lambda x: len(str(x).split(" ")))

# Character counts
g_df['character_count_sml'] = g_df['lyrics_sml'].apply(lambda x: sum(len(word) for word in str(x).split(" ")))

#average word length
g_df['avg_word_length_sml'] = g_df['character_count_sml'] / g_df['word_count_sml']

CPU times: user 1.28 s, sys: 38.2 ms, total: 1.32 s
Wall time: 1.32 s


In [33]:
g_df.describe(include='all')

,original_csv,artist_name,song_name,link,lyrics,language,genre,date,spaCy_Doc,word_count,character_count,avg_word_length,lyrics_sml,spaCy_Doc_sml,word_count_sml,character_count_sml,avg_word_length_sml
count,86391,86391,86391,86391,86391,86391,86391,0.0,86391,86391.000000,86391.000000,86391.000000,86391,86391,86391.000000,86391.000000,86391.000000
unique,1,1147,66856,86391,86309,1,6,NaN,86391,NaN,NaN,NaN,85463,86391,NaN,NaN,NaN
top,genres_csv,elvis-presley,Intro,/eric-clapton/better-make-it-through-today.html,"so, so you think you can tell. heaven from hel...",ENGLISH,Rock,NaN,"(it, is, been, a, long, ,, long, while, ., sin...",NaN,NaN,NaN,merry little christmas let heart light trouble...,"(whoow, !, here, is, a, little, song, for, eve...",NaN,NaN,NaN
freq,86391,747,50,1,3,86391,47408,NaN,1,NaN,NaN,NaN,4,1,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,289.744059,1132.871051,3.942745,NaN,NaN,108.420032,552.281418,5.120008
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,177.241573,691.961143,0.376259,NaN,NaN,74.626140,379.383164,0.468566
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,8.000000,1.740741,NaN,NaN,1.000000,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,171.000000,677.000000,3.719021,NaN,NaN,61.000000,312.000000,4.812500
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,245.000000,961.000000,3.903427,NaN,NaN,88.000000,450.000000,5.083333
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,357.000000,1381.000000,4.116123,NaN,NaN,129.000000,661.000000,5.384615


In [34]:
!touch g_spaCy2_df.csv
g_df.to_csv('g_spaCy2_df.csv', index= False)
!aws s3 cp g_spaCy2_df.csv s3://music-lyrics-chain/g_spaCy2_df.csv
%time

upload: ./g_spaCy2_df.csv to s3://music-lyrics-chain/g_spaCy2_df.csv
CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.15 µs


In [36]:
g_df.loc[84304]#genres Taylor Swift The Story of Us

original_csv                                                  genres_csv
artist_name                                                 taylor-swift
song_name                                                The Story Of Us
link                                  /taylor-swift/the-story-of-us.html
lyrics                 i used to think one day we would tell the stor...
language                                                         ENGLISH
genre                                                                Pop
date                                                                 NaN
spaCy_Doc              (i, used, to, think, one, day, we, would, tell...
word_count                                                           484
character_count                                                     1764
avg_word_length                                                 3.644628
lyrics_sml             think day tell story met sparks flew instantly...
spaCy_Doc_sml          (i, used, to, think, one, da